For executing the wavelet based fusion on 16-bit images.
Fisrt split 16-bit image into two 8-bit images.
So, for each energy image there will be two 8-bit images. So, do the image using corresponding images.

In [25]:
import cv2
import numpy as np

img_16 = cv2.imread(r'D:\Downloads\Bag_dumps-20230428T040901Z-001\Bag_dumps\Dumps\denoised\final\P01X010-2022111700190_low_final.png', -1)

img_8_1 = (img_16 >> 8).astype(np.uint8)
img_8_2 = (img_16 & 0xFF).astype(np.uint8)

cv2.imwrite('output_image1.png', img_8_1)
cv2.imwrite('output_image2.png', img_8_2)


True

Now apply wavelet fusion

In [10]:
import pywt
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the low energy and high energy images
lowE_img = cv2.imread(r'D:\Downloads\Bag_dumps-20230428T040901Z-001\Bag_dumps\Dumps\denoised\final\P01X010-2022111700190_low_final.png', 0)
highE_img = cv2.imread(r'D:\Downloads\Bag_dumps-20230428T040901Z-001\Bag_dumps\Dumps\denoised\final\P01X010-2022111700190_high_final.png', 0)

# Define wavelet function
wavelet = 'db4'

# Perform 2D discrete wavelet transform on low energy image
lowE_coeffs = pywt.wavedec2(lowE_img, wavelet, level=4)
# Unpack coefficients
LcA4, (LcH4, LcV4, LcD4), (LcH3, LcV3, LcD3), (LcH2, LcV2, LcD2), (LcH1, LcV1, LcD1) = lowE_coeffs


# Perform 2D discrete wavelet transform on high energy image
highE_coeffs = pywt.wavedec2(highE_img, wavelet, level=4)
# Unpack coefficients
HcA4, (HcH4, HcV4, HcD4), (HcH3, HcV3, HcD3), (HcH2, HcV2, HcD2), (HcH1, HcV1, HcD1) = highE_coeffs

In [11]:
# Apply a low-pass filter to the approximation coefficients of low energy image
kernel = np.array([
    [0.077847, 0.123317, 0.077847],
    [0.123317, 0.195346, 0.123317],
    [0.077847, 0.123317, 0.077847]])

lowE_A_coeffs_filtered = cv2.filter2D(LcA4, -1, kernel)

# Apply a low-pass filter to the approximation coefficients of high energy image
highE_A_coeffs_filtered = cv2.filter2D(HcA4, -1, kernel)

In [12]:
fused_A_coeffs = (lowE_A_coeffs_filtered/2) + (highE_A_coeffs_filtered/2)
#fused_A_coeffs = (lowE_A_coeffs_filtered + highE_A_coeffs_filtered)/2

In [13]:
# Add low energy and high energy detail coefficients together
new_detail_coeffs = [(LcH4 + HcH4, LcV4 + HcV4, LcD4 + HcD4),
                     (LcH3 + HcH3, LcV3 + HcV3, LcD3 + HcD3),
                     (LcH2 + HcH2, LcV2 + HcV2, LcD2 + HcD2),
                     (LcH1 + HcH1, LcV1 + HcV1, LcD1 + HcD1)]

In [16]:
# Reconstruct the new image from the modified coefficients
new_img = pywt.waverec2([fused_A_coeffs, *new_detail_coeffs], wavelet)

# Save the new image
cv2.imwrite('output.png', new_img)

True

This is the code to combine 8-bit images to form 16-bit images

In [1]:
import cv2
import numpy as np

img_8_1 = cv2.imread(r"C:\Users\ML08\Desktop\CONVENTIONAL\Poisson\denoised_image_ML.png", cv2.IMREAD_GRAYSCALE)
img_8_2 = cv2.imread(r"C:\Users\ML08\Desktop\CONVENTIONAL\Poisson\denoised_image_LL.png", cv2.IMREAD_GRAYSCALE)

img_16_1 = img_8_1.astype(np.uint16) << 8
img_16_2 = img_8_2.astype(np.uint16)

img_16 = img_16_1 | img_16_2

cv2.imwrite('output_image.png', img_16)


True